In [2]:
#Bibliotecas a importar
import numpy as np
import math as math
import itertools

### Sesión 4: Valuación de Opciones en tiempo discreto

In [67]:
#t_op: Tipo de opcion
# 1: Opción europea tipo call
# 2: Opción europea tipo put
# 3: Opción digital tipo call
# 4: Opción digital tipo put
# 5: Opción asiática
# 6: Opción lookback
# 7: Opción Barrera Up&Out
# 8: Opción Barrera Up&In
# 9: Opción Americana call
# 10: Opción Americana put
t_op = 10
T = 1 #Tiempo de vigencia de la opción (en años)
r = 0.056 #tasa anual libre de riesgo
k = 5 #Precio strike. Funciona como el salto en el caso de las digitales
S_0 = 4 #Precio del subyacente a t=0
n = 2 #Particiones del árbol
sigma= 0 #Volatilidad. Si es diferente a 0 calculamos u y d como en el caso contínuo
reb = 2 #Rebate
b = 5 #Barrera
u = 2 #Factor de aumento en el precio
d = 0.5 #Factor de disminución de precio

# Probabilidad en la medida libre de riesgo

In [68]:
delta_t = T / n
if (sigma!=0):
    u=math.exp((r-(sigma**2)/(2))*delta_t+sigma*math.sqrt(delta_t))
    d=math.exp((r-(sigma**2)/(2))*delta_t-sigma*math.sqrt(delta_t))
p = (math.exp(r * delta_t) - d) / (u - d) #Probabilidad de aumento en el precio

## Construcción del árbol

### Subyacente

Valor en el primer paso

In [70]:
#Matriz que almacena los valores del subyacente
arbol = np.zeros((n + 1, n + 1))
#El primer valor de la matriz es el subyacente a precio en t=0
arbol[0,0] = S_0
arbol #Visualziamos

array([[4., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

Valor del subyacente en el resto de los pasos

In [71]:
#For para llenar los valores de la matriz
for col in range(1, n +1):
    for ren in range(0, n +1):
        #Condicional para limitar matriz superior triangular
        if((col - ren) >= 0):
            arbol[ren, col] = S_0 *(( u ** (col - ren)) * (d ** (ren)))
arbol #Visualizamos

array([[ 4.,  8., 16.],
       [ 0.,  2.,  4.],
       [ 0.,  0.,  1.]])

In [72]:
if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    arbol_aux=np.zeros((2**n, n+1))
    for col in range(0,n+1):
        if(col==0):
            arbol_aux[0,0]=S_0
        else:
            up_down=list(itertools.product(['u','d'],repeat=col))
            for ren in range(0,2**col):
                bajas=0
                pos=up_down[ren]
                for letter in pos:
                    if (letter=='d'):
                        bajas=bajas+1
                arbol_aux[ren,col]=S_0*(u**(col-bajas))*d**(bajas)

### Payoff

Primer paso

In [73]:
#Matriz que almacena el valor de los payoffs
payoffs = np.zeros((n+1, n+1))
#Sobreescribimos la última columna de la matriz de PAYOFSS
#IMPORTANTE: Agregar condicional para el caso del put (este es el caso particular del call)
if (t_op==1 or t_op==9):
    for ren in range(0, n+1):
        payoffs[ren, n] = max(arbol[ren, n] - k, 0)
elif (t_op==2 or t_op==10):
    for ren in range(0, n+1):
        payoffs[ren, n] = max(k-arbol[ren, n] , 0)
elif t_op==3:
    for ren in range(0, n+1):
        if (arbol[ren, n] > S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==4:
    for ren in range(0, n+1):
        if (arbol[ren, n] < S_0):
            payoffs[ren, n] = k
        else:
            payoffs[ren, n] = 0
elif t_op==5:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        suma=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            suma=suma+arbol[bajas,(i+1)]
        payoffs[ren,n]=max((suma/(n+1))-k,0)
elif t_op==6:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        payoffs[ren,n]=max(valores)
elif t_op==7:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        if(max(valores)>=b):
            payoffs[ren,n]=reb
        else:
            payoffs[ren,n]=max(arbol_aux[ren, n] - k, 0)
elif t_op==8:
    payoffs=np.zeros((2**n, n+1))
    up_down=list(itertools.product(['u','d'],repeat=n))
    for ren in range(0,2**n):
        pos=up_down[ren]
        valores=np.zeros(n+1)
        valores[0]=S_0
        bajas=0
        for i in range(0,n):
            if(pos[i]=='d'):
                bajas=bajas+1
            valores[i+1]=arbol[bajas,(i+1)]
        if(max(valores)<=b):
            payoffs[ren,n]=reb
        else:
            payoffs[ren,n]=max(k-arbol_aux[ren, n], 0)
payoffs #Visualizamos

array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 4.]])

Valuamos backwards

In [76]:
#Sobreescribimos matriz de payoffs mediante el método iterativo
if(t_op!=5):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1])

if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    for i in range(1, n+1):
        col = n - i
        for j in range(0, 2**col):
            payoffs[j, col] = math.exp(-r * delta_t) * (p * payoffs[2*j, col + 1] + (1 - p)*payoffs[2*j +1, col +1])
if(t_op==9):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = max(arbol[j,col]-k,math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1]))
if(t_op==10):
    for i in range(1, n + 1):
        col = n - i
        for j in range(0, col +1):
            payoffs[j, col] = max(k-arbol[j,col],math.exp(-r * delta_t) * (p * payoffs[j, col + 1] + (1 - p)*payoffs[j +1, col +1]))
payoffs #visualizamos

array([[1.69496733, 0.48619418, 0.        ],
       [0.        , 3.        , 1.        ],
       [0.        , 0.        , 4.        ]])

## Valor de la opción

In [60]:
payoffs[0,0]

0.5423118291444479

## Composición del portafolio

# Alfas

In [77]:
#Matriz que almacena los valores de las alfas
alfas = np.zeros((n, n ))
if(t_op!=5):
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                alfas[ren,col]=(payoffs[ren,col+1]-payoffs[ren+1,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1])
if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    alfas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
            alfas[ren,col]=(payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1])
alfas #Visualizamos

array([[-0.41896764, -0.08333333],
       [ 0.        , -1.        ]])

# Betas

In [62]:
#Matriz que almacena los valores de las alfas
if(t_op!=5):
    betas = np.zeros((n, n ))
    for ren in range(0,n):
        for col in range(0, n):
            #Condicional para limitar matriz superior triangular
            if((col - ren) >= 0):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[ren,col+1]-((payoffs[ren,col+1]-payoffs[ren+1,col+1])*arbol[ren,col+1])/(arbol[ren,col+1]-arbol[ren+1,col+1]))

if(t_op==5 or t_op==6 or t_op==7 or t_op==8):
    betas = np.zeros((2**n, n))
    for col in range(0,n):
    #Condicional para limitar matriz superior triangular
        for ren in range(0, 2**col):
                betas[ren,col]=math.exp(-r * delta_t)*(payoffs[2*ren,col+1]-((payoffs[2*ren,col+1]-payoffs[2*ren+1,col+1])*arbol_aux[2*ren,col+1])/(arbol_aux[2*ren,col+1]-arbol_aux[2*ren+1,col+1]))
betas #Visualizamos

array([[-2.82372016, -5.37049175],
       [ 0.        , -0.43614692],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ]])